In [51]:
import time
from multiprocessing import Process
import os

processes = list()
num_processes = os.cpu_count()

# create processes
function = lambda x: print(x ** 2)
for i in range(num_processes):
    # target is the function to run by the process
    p = Process(target=function(i))
    # add the process to the list of processes
    processes.append(p)

# start the process
# I am using list comprehension just for fun
[p.start() for p in processes]

# Each processes should wait others until they finish
# Here I am blocking the main thread until everything is done
[p.join() for p in processes]

0
1
4
9


[None, None, None, None]

In [60]:
# Processes don't share memory space
# So you can't simply define a variable as a global to be shared among prcesses
# We can use Value for single value, Array for multilple values
from multiprocessing import Value, Lock
import time

# 'i' means the data type is integer
# 0 is the starting value

shared_number = Value('i', 0)
print(f'Number at beginning is {shared_number.value}')

lock = Lock()


def add_100(sh_number, lock):
    # we are adding this delay so the scheduler switches to another process
    for i in range(100):
        time.sleep(0.001)
        # creating critical section
        # using lock in this way is better than using acquire and release
        with lock:
            sh_number.value += 1


p1 = Process(target=add_100(shared_number, lock))
p2 = Process(target=add_100(shared_number, lock))

p1.start()
p2.start()

p1.join()
p2.join()
print(f'Number at end is {shared_number.value}')

Number at beginning is 0
Number at end is 200


In [64]:
from multiprocessing import Array

# type is double
shared_array = Array('d', [0.0, 100.0, 200.0])
print(f'Array at beginning is {shared_array[:]}')
lock = Lock()


def add_100(sh_array, lock):
    # we are adding this delay so the scheduler switches to another process
    for i in range(100):
        time.sleep(0.01)
        # creating critical section
        # using lock in this way is better than using acquire and release
        for j in range(len(shared_array)):
            with lock:
                sh_array[j] += 1


p1 = Process(target=add_100(shared_array, lock))
p2 = Process(target=add_100(shared_array, lock))

p1.start()
p2.start()

p1.join()
p2.join()
print(f'Array at beginning is {shared_array[:]}')

Array at beginning is [0.0, 100.0, 200.0]
Array at beginning is [200.0, 300.0, 400.0]


In [71]:
from multiprocessing import Queue

q = Queue()


# Q is used for inter-processes actions

def square(numbers, q):
    for i in numbers:
        q.put(i * i)


def negative(numbers, q):
    for i in numbers:
        q.put(i * -1)


numbers = [1, 2, 3, 4, 5]
p1 = Process(target=square(numbers, q))
p2 = Process(target=negative(numbers, q))

p1.start()
p2.start()

p1.join()
p2.join()

while not q.empty():
    print(q.get())

1
4
9
16
25
-1
-2
-3
-4
-5


In [80]:
# Process pool is used to manage multiple process
from multiprocessing import Pool


def cube(number):
    return number ** 3


numbers = range(10)

pool = Pool()
# will create # of process as your machine have and split the iterable between processes
result = pool.map(func=cube, iterable=numbers)
# closes accepting actions
pool.close()
# wait for processes
pool.join()
print(result)
